<a href="https://colab.research.google.com/github/leegyuhi/oooooooooooooh/blob/master/jtp_Ch_7_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ch.7 합성곱 신경망(CNN)
- 딥러닝을 활용한 이미지 인식 분야에서 활용

## 7.1 전체 구조
[그림 7-1]
- 완전연결 신경망 : 'Affine-ReLU'
- CNN : 'Conv-ReLU-Pooling'
  - 합성곱 계층(convolutional layer)
  - 풀링 계층(pooling layer)
1. 출력에 가까운 층에서는 'Affine-ReLU' 구성 사용
2. 마지막 출력 계층에서는 'Affine-Softmax' 구성 사용

## 7.2 합성곱 계층

### 7.2.1 완전연결 계층의 문제점
- 데이터의 형상이 무시된다.
  - 3차원 데이터 $\rightarrow$ 1차원 데이터로 평탄화
  - MNIST 데이터셋 : 28$\times$28 $\rightarrow$ 784$\times$1
  - 3차원 형상에 담긴 정보 무시
- 합성곱 계층 : 3차원 데이터 입력 $\rightarrow$ 다음 층에 3차원 데이터 전달
  - 입력 특징 맵(input feature map) : 합성곱 계층의 입력 데이터
  - 출력 특징 맵(output feature map) : 합성곱 계층의 출력 데이터

### 7.2.2 합성곱 연산
- 합성곱 연산(필터 연산)
  1. 3$\times$3 윈도우를 일정 간격으로 이동해가며 입력 데이터에 적용
  2. 단일 곱셈-누산 : 입력과 필터에서 대응하는 원소끼리 곱한 후 그 총합
  3. 결과를 출력의 해당 장소에 저장

  $$\begin{bmatrix}1 &2  &3  &0 \\ 0 &1  &2  &3 \\ 3 &0  &1  &2 \\ 2 &3  &0  &1 \end{bmatrix}
\circledast  \begin{bmatrix}2 &0  &1 \\ 0 &1  &2 \\ 1 &0  &2 \end{bmatrix}
= \begin{bmatrix}15 &16 \\ 6 &15 \end{bmatrix}$$


$$\begin{bmatrix}1 &2  &3 \\ 0 &1  &2 \\ 3 &0  &1 \end{bmatrix}
\circledast \begin{bmatrix}2 &0  &1 \\ 0 &1  &2 \\ 1 &0  &2 \end{bmatrix}
= \begin{bmatrix}15 & \\  & \end{bmatrix}$$

$$1\cdot 2+2\cdot 0+3\cdot 1+0\cdot 0+1\cdot 1+2\cdot 2+3\cdot 1+0\cdot 0+1\cdot 2=15$$

In [4]:
import numpy as np
from scipy import signal

in1=np.array([[1,2,3,0],[0,1,2,3],[3,0,1,2],[2,3,0,1]])
in2=np.array([[2,0,1],[0,1,2],[1,0,2]])
convolution=scipy.signal.correlate2d(in1, in2, mode='valid', boundary='fill', fillvalue=0); convolution

array([[15, 16],
       [ 6, 15]])

- CNN에 편향 존재
  - 편향은 필터를 적용한 후의 데이터에 더해진다.
  
  $$\begin{bmatrix}1 &2  &3  &0 \\ 0 &1  &2  &3 \\ 3 &0  &1  &2 \\ 2 &3  &0  &1 \end{bmatrix}
\circledast  \begin{bmatrix}2 &0  &1 \\ 0 &1  &2 \\ 1 &0  &2 \end{bmatrix}
\rightarrow  \begin{bmatrix}15 &16 \\ 6 &15 \end{bmatrix}+\begin{bmatrix}3\end{bmatrix}
\rightarrow \begin{bmatrix}18 &19 \\ 9 &18 \end{bmatrix}$$
  - 편향은 하나(1$\times$1)만 존재

### 7.2.3 패딩
- 패딩(padding) : 입력 데이터 주변을 특정 값(0)으로 채우고 합성곱 연산 수행
- (4,4) 크기의 입력 데이터에 폭이 1인 패딩 적용

In [0]:
in1=np.array([[1,2,3,0],[0,1,2,3],[3,0,1,2],[2,3,0,1]])
in2=np.array([[2,0,1],[0,1,2],[1,0,2]])
convolution=scipy.signal.correlate2d(in1, in2, mode='same', boundary='fill', fillvalue=0); convolution

array([[ 7, 12, 10,  2],
       [ 4, 15, 16, 10],
       [10,  6, 15,  6],
       [ 8, 10,  4,  3]])

  - 출력의 크기 조정을 위해 사용
  - 패딩의 폭을 1로 설정하면, 입력 데이터의 공간적 크기를 고정한 채 다음 계층에 전달 가능

### 7.2.4 스트라이드
- 스트라이드(stride) : 필터를 적용하는 위치의 간격
  - 스트라이드를 2로하면 필터를 적용하는 윈도우가 두 칸씩 이동한다.
  - 스트라이드를 키우면 출력의 크기는 작아진다.
- 입력 크기 (H,W), 필터 크기 (FH,FW), 출력 크기 (OH,OW), 패딩 P, 스트라이드 S

  $$OH=\frac{H+2P-FH}{S}+1,\: OW=\frac{W+2P-FW}{S}+1$$
  - ex) 입력 (4,4), 패딩 1, 스트라이드 1, 필터 (3,3)

  $$OH=\frac{4+2\cdot 1-3}{1}+1=4,\: OW=\frac{4+2\cdot 1-3}{1}+1=4$$

  - OH와 OW가 정수로 나눠떨어지는 값이어야 한다.

### 7.2.5 3차원 데이터의 합성곱 연산
- 채널을 고려한 3차원 데이터를 다루는 합성곱 연산
- 입력 데이터와 필터의 합성곱 연산을 채널마다 수행하고, 그 결과를 더하여 하나의 출력을 구함 [그림 7-9]
  - 입력 데이터의 채널 수 = 필터의 채널 수
  - 모든 채널의 필터 크기 동일

### 7.2.6 블록으로 생각하기
- 3차원 데이터 = 직육면체 블록 (채널, 높이, 너비)
  - 입력 데이터 (C, H, W)
  - 필터 (C, FH, FW)
  - (C, H, W) $\circledast$ (C, FH, FW) = (1, OH, OW)
- 합성곱 연산의 출력으로 다수의 채널을 내보내기 위해 필터(가중치)를 다수 사용
  - 필터 FN개 적용 $\rightarrow$ 출력 맵 FN개 생성
  - (C, H, W) $\circledast$ (FN, C, FH, FW) = (FN, OH, OW) : CNN의 처리 흐름
  - 필터 (출력 채널 수, 입력 채널 수, 높이, 너비) : 4차원 데이터
- 편향 (FN, 1,1)
  - (C, H, W) $\circledast$ (FN, C, FH, FW) $\rightarrow$ (FN, OH, OW) + (FN, 1,1) = (FN, OH, OW)
  - 브로드캐스트 : 형상이 다른 블록의 덧셈

### 7.2.7 배치 처리
- 4차원 데이터 (데이터 수, 채널 수, 높이, 너비)
- 데이터가 N개일 때,
  - (N, C, H, W) $\circledast$ (FN, C, FH, FW) $\rightarrow$ (N, FN, OH, OW) + (FN, 1,1) = (N, FN, OH, OW)
- N회 분의 합성곱 연산을 한 번에 수행

## 7.3 풀링 계층
- 풀링 : 세로, 가로 방향의 공간을 줄이는 연산
- 2$\times$2 최대 풀링을 스트라이드 2로 처리

$$\begin{bmatrix}1 &2  &1  &0 \\ 0 &1  &2  &3 \\ 3 &0  &1  &2 \\ 2 &4  &0  &1 \end{bmatrix}\rightarrow 
\begin{bmatrix}2 &3 \\ 4 &2 \end{bmatrix}$$
- 일반적으로 풀링의 윈도우 크기와 스트라이드를 같은 값으로 설정
- cf) 평균 풀링 : 대상 영역의 평균을 계산

$$\begin{bmatrix}1 &2  &1  &0 \\ 0 &1  &2  &3 \\ 3 &0  &1  &2 \\ 2 &4  &0  &1 \end{bmatrix}\rightarrow 
\begin{bmatrix}1 &1.5 \\ 2.25 &1 \end{bmatrix}$$

### 7.3.1 풀링 계층의 특징
1.  학습해야 할 매개변수가 없다.
2.  채널 수가 변하지 않는다.
3. 입력의 변화에 영향을 적게 받는다. 

## 7.4 합성곱/풀링 계층 구현하기

### 7.4.1 4차원 배열
- 입출력 데이터가 4차원 데이터

In [0]:
# 높이 28, 너비 28, 채널 1개인 데이터 10개
x = np.random.rand(10,1,28,28) #무작위로 데이터 생성
x.shape

(10, 1, 28, 28)

In [0]:
x[0].shape #10개 중 첫 번째 데이터

(1, 28, 28)

In [0]:
x[1].shape #10개 중 두 번째 데이터

(1, 28, 28)

In [0]:
x[0][0].shape #첫 번째 데이터의 첫 채널의 공간 데이터

(28, 28)

### 7.4.2 im2col로 데이터 전개하기
- im2col : 입력 데이터를 필터링하기 좋게 전개하는 함수
  - 3차원 입력 데이터 $\rightarrow$ 2차원 행렬 [그림7-18]
    1. 입력 데이터에서 필터를 적용하는 영역(3차원 블록)을 한 줄로 배열
    2. 1의 전개를 필터를 적용하는 모든 영역에서 수행
- 필터 적용 영역이 겹치게 되면 im2col로 전개한 후의 원소 수가 원래 원소 수보다 많아진다.
- 선형 대수 라이브러리는 행력 계산에 고도로 최적화되어 큰 행력의 곱셈을 빠르게 계산 할 수 있다. 
- im2col을 이용한 데이터 전개 방법 [그림7-19]
  1. im2col로 입력 데이터 전개
  2. 합성곱 계층의 필터를 1열로 전개
  3. 두 행렬의 곱 계산
  4. 2차원인 출력 데이터를 4차원으로 변형

### 7.4.3 합성곱 계층 구현하기
- im2col 함수의 인터페이스

In [0]:
im2col(input_data, filter_h, filter_w, stride=1, pad=0)

In [6]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1,3,7,7) #(데이터 수, 채널 수, 너비, 높이)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10,3,7,7) #데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


- im2col을 사용한 합성곱 계층의 구현

In [0]:
class Convolution:
  def __init__(self, w, b, stride=1, pad=0): #인수 초기화
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad
  
  def forward(self, x):
    FN, C, FH, FW = self.W.shape #필터
    N, C, H, W = x.shape #입력데이터
    out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
    out_w = int(1 + (W + 2*self.pad - FW) / self.stride)

    col = im2col(x, FH, FW, self.stride, self.pad) #입력 데이터를 im2col으로 전개
    col_W = self.W.reshape(FN, -1).T #필터를 reshape 사용하여 2차원 배열로 전개
    out = np.dot(col, col_W) + self.b #전개한 두 행렬의 곱 + 편향

    out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2) #transpose를 사용하여 출력데이터의 축 순서 변경

    return out

### 7.4.4 풀링 계층 구현하기

In [0]:
class Pooling:
  def __init__(self, pool_h, pool_w, stride=1, pad=0):
    self.pool_h = pool_h
    self.pool_w = pool_w
    self.stride = stride
    self.pad = pad

  def forward(self, x):
    N, C, H, W = x.shape
    out_h = int(1 + (H - self.pool_h) / self.stride)
    out_w = int(1 + (W - self.pool.w) / self.stride)

    #전개
    col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    col = col.reshape(-1, self.pool_h * self.pool_w)

    #최댓값
    out = np.max(col, axis=1) #행별 최댓값 계산

    #성형
    out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)

    return out

- 풀링 계층 구현
  1. 입력 데이터 전개
  2. 행별 최댓값
  3. 적절한 모양으로 성형